In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
!ls -ltr
!hostname -i

total 167064
drwxr-xr-x 2 abhinav abhinav      4096 Jun  5  2009 cifar-10-batches-py
drwxr-xr-x 3 abhinav abhinav      4096 Sep 25 21:56 data_augmentation
-rw-r--r-- 1 abhinav abhinav        24 Sep 25 21:56 readme.md
-rw-r--r-- 1 abhinav abhinav     11803 Sep 25 23:28 model_architecture
-rw-r--r-- 1 abhinav abhinav    153822 Sep 26 13:03 Test_missclassified_images.jpg
-rw-r--r-- 1 abhinav abhinav     41673 Sep 26 14:29 model_history.png
-rw-r--r-- 1 abhinav abhinav    174027 Sep 26 14:30 QuizDNN.ipynb
drwxr-xr-x 4 abhinav abhinav      4096 Sep 29 05:24 model_file
-rw-r--r-- 1 abhinav abhinav 170498071 Sep 29 05:46 cifar-10-python.tar.gz
drwxr-xr-x 3 abhinav abhinav      4096 Sep 29 05:48 lr_finder
drwxr-xr-x 4 abhinav abhinav      4096 Sep 29 05:58 mlruns
-rw-r--r-- 1 abhinav abhinav    134974 Sep 30 04:45 EVA5_session10.ipynb
drwxr-xr-x 4 abhinav abhinav      4096 Sep 30 04:46 model_utility
-rw-r--r-- 1 abhinav abhinav     17401 Sep 30 04:46 EVA5_session9.ipynb
127.0.1.1 


# Loading UD modules

In [3]:
from __future__ import print_function
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from tqdm import tqdm
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import os 
import numpy as np
#mlflow
import mlflow
import mlflow.pytorch

# CUSTOM LIBRARY
import model_utility.data_utils as dutils
import model_utility.model_utils as mutils
import model_utility.plot_utils as putils 
import model_utility.regularization as regularization
import model_file.model_cifar as model_cifar
import model_utility.alb_utils as alb
import sys
sys.path.append('/home/abhinav/Documents/EVA5/session9/model_utility/gradcam')
import model_utility.grad_cam_viz as viz
import model_file.models as mod

# PLOTS
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import data_augmentation.albumentations as A
from data_augmentation.albumentations.pytorch import ToTensor

In [5]:
A.__version__
A.__file__
A.Compose.__module__
A.Cutout.__module__
A.RandomCrop.__module__
A.HorizontalFlip.__module__
A.ElasticTransform.__module__
A.CoarseDropout.__module__
A.Normalize.__module__
A.pytorch.ToTensor.__module__
A.pytorch.ToTensor.__module__

'0.4.6'

'/home/abhinav/Documents/GitHub/Eva5/session10/data_augmentation/albumentations/__init__.py'

'data_augmentation.albumentations.core.composition'

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.augmentations.transforms'

...

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.pytorch.transforms'

'data_augmentation.albumentations.pytorch.transforms'

# Check GPU avaiability

In [6]:
device = dutils.get_device()

CUDA Available? True
Device is cuda:0


In [7]:
cuda = torch.cuda.is_available()
print(cuda)

True


# CIFAR10 Basics

In [8]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

SEED = 1
batch_size=128

# CUDA?
if device.type == 'cuda':
    torch.cuda.manual_seed(SEED)
else:
    torch.manual_seed(SEED)

# Load CIFAR10 dataset

In [9]:
os.getcwd()

'/home/abhinav/Documents/GitHub/Eva5/session10'

In [10]:
path = os.getcwd()
path

'/home/abhinav/Documents/GitHub/Eva5/session10'

In [11]:
batch_size = 128

In [12]:
# load CIFAR10 dataset
trainset, testset, train_loader, test_loader = alb.get_dataloader(batch_size=batch_size, num_workers=4, cuda = True,path = path)

Running over Cuda !!  True
Files already downloaded and verified
Files already downloaded and verified
mean 0.49186864
standard dev 0.24697131
mean 0.48265335
standard dev 0.24338901
mean 0.44717798
standard dev 0.26159254


/home/abhinav/anaconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1469: DeprecationWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn("This class has been deprecated. Please use CoarseDropout", DeprecationWarning)
/home/abhinav/anaconda3/lib/python3.8/site-packages/albumentations/pytorch/transforms.py:57: DeprecationWarning: ToTensor is deprecated and will be replaced by ToTensorV2 in albumentations 0.5.0
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


# Data mage Visualization

In [13]:
#Plot 20 random images
putils.plot_dataset_images(device, classes, train_loader)

RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "/home/abhinav/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 31, in _pin_memory_loop
    data = pin_memory(data)
  File "/home/abhinav/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in pin_memory
    return [pin_memory(sample) for sample in data]
  File "/home/abhinav/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in <listcomp>
    return [pin_memory(sample) for sample in data]
  File "/home/abhinav/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 47, in pin_memory
    return data.pin_memory()
RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1595629395347/work/aten/src/THC/THCCachingHostAllocator.cpp:278


<Figure size 720x648 with 0 Axes>

### Model stracture

In [ ]:
model = mod.ResNet18()

In [ ]:
mutils.model_summary(model, device, input_size=(3, 32, 32))

# Visual Summary

In [ ]:
# from model_utility.plot_utils import  graphical_summary_cifar10
# graphical_summary_cifar10(model)

In [ ]:
# Create CIFAR10 network
# model = model_cifar.Net(dropout_value = 0.05)
# mutils.model_summary(model, device, input_size=(3, 32, 32))

# Build and train model

In [ ]:
from tqdm import tqdm

In [ ]:
# build and train the model
model_name = "Model CIFAR10"
EPOCHS = 20
model =  model.to(device) # use GPU

# using L1 and L2 regularization
train_acc, train_losses, test_acc, test_losses = mutils.build_model(model, 
                                                            device, 
                                                            trainloader=train_loader, 
                                                            testloader=test_loader, 
                                                            epochs=EPOCHS,
                                                            L1_loss_flag=False,
                                                            L2_penalty_val=0.001)

In [ ]:
A.__file__

# Test Accuracy

In [ ]:
mutils.get_test_accuracy(model, device, test_loader)

### Class level Accuracy

In [ ]:

mutils.class_based_accuracy(model, device, classes, test_loader)

# Visualization

In [ ]:
print("\nPloting Accuracy and Loss history: {}\n".format(model_name))
putils.plot_model_history(model_name, 
                   train_acc,test_acc,
                   train_losses,test_losses,
                   save_filename = "model_history")

# Misclassified images

In [ ]:
print ("\nPloting misclassified images: {}\n".format(model_name))

putils.miss_classification('Test',model, 
                          device, 
                          classes,
                          testloader=test_loader,
                          num_of_images = 20,
                          save_filename="model_misclassified")

In [ ]:
layer = 'layer4'
viz.gradcam_plot(layer,model,testset,classes)

In [ ]:
with mlflow.start_run() as run:
    mlflow.log_param("epochs", EPOCHS)
    mlflow.pytorch.log_model(model, "models")